In [33]:
#Import required libraries for data loading, cleaning and pre processing.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [34]:
#Load the dataset
df = pd.read_csv('train.csv')
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
1,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5.0,NaN,3.50



**The dataset consists of 5847 rows and 14 columns**

In [35]:
#Descriptive Statistics
print(df.info())
print(df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5847 entries, 0 to 5846
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         5847 non-null   int64  
 1   Name               5847 non-null   object 
 2   Location           5847 non-null   object 
 3   Year               5847 non-null   int64  
 4   Kilometers_Driven  5847 non-null   int64  
 5   Fuel_Type          5847 non-null   object 
 6   Transmission       5847 non-null   object 
 7   Owner_Type         5847 non-null   object 
 8   Mileage            5845 non-null   object 
 9   Engine             5811 non-null   object 
 10  Power              5811 non-null   object 
 11  Seats              5809 non-null   float64
 12  New_Price          815 non-null    object 
 13  Price              5847 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 639.6+ KB
None
        Unnamed: 0         Year  Kilometers_Driven      

## **a) Handling Missing Values**

In [36]:
df.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,2
Engine,36


Here's a summary of the missing values:

1. Mileage: 2 missing values
2. Engine: 36 missing values
3. Power: 36 missing values
4. Seats: 38 missing values
5. New_Price: 5032 missing values (a majority of this column is missing)

For each column:

**1. Mileage, Engine, Power, Seats:** These columns have relatively few missing values compared to the total number of entries. We can impute them with the median, as these are numerical values, and the median is less affected by outliers.

**2. New_Price:** Since this column has over 86% missing data, it's best to drop it, as imputing would not provide meaningful results.

In [37]:
# Extract numeric values and convert the columns - THIS IS STEP B
df['Mileage'] = df['Mileage'].str.extract(r'(\d+\.\d+|\d+)').astype(float)  # Remove non-numeric parts and convert to float
df['Engine'] = df['Engine'].str.extract(r'(\d+)').astype(float)
df['Power'] = df['Power'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

In [38]:
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")

# Impute median for Mileage, Engine, Power, Seats
df['Mileage'].fillna(df['Mileage'].median(), inplace=True)
df['Engine'].fillna(df['Engine'].median(), inplace=True)
df['Power'].fillna(df['Power'].median(), inplace=True)
df['Seats'].fillna(df['Seats'].median(), inplace=True)

In [39]:
# Drop New_Price due to a high number of missing values
df.drop(columns=['New_Price'], inplace=True)

In [40]:
df.isnull().sum()

,0
Unnamed: 0,0
Name,0
Location,0
Year,0
Kilometers_Driven,0
Fuel_Type,0
Transmission,0
Owner_Type,0
Mileage,0
Engine,0


**We can see that there are no missing values after being imputed.**

## **Step b: Remove units has already been done when extracting numeric values above**


## **C) One hot encoding categorical features**

1. Fuel_Type and Transmission were converted into numerical one-hot encoded columns.

In [41]:
#One-hot encoding categorical variables
df = pd.get_dummies(df, columns=['Fuel_Type', 'Transmission'], drop_first=True)
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,False,False,True
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,True,True
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,False,False,False
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,False,False,True


## **d) Creating a New Feature:**
A new column Car_Age was added, representing the car's age (2024 - Year).

In [43]:
#Creating a new feature 'Car_Age'
df['Car_Age'] = 2024 - df['Year']
df.head()

,Unnamed: 0,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Fuel_Type_Electric,Fuel_Type_Petrol,Transmission_Manual,Car_Age
0,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582.0,126.20,5.0,12.50,False,False,True,9
1,2,Honda Jazz V,Chennai,2011,46000,First,13.00,1199.0,88.70,5.0,4.50,False,True,True,13
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248.0,88.76,7.0,6.00,False,False,True,12
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968.0,140.80,5.0,17.74,False,False,False,11
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461.0,63.10,5.0,3.50,False,False,True,11


## **e) SQL-Like Operations:**
**1. Select:** Columns selected include 'Name', 'Location', 'Price', 'Car_Age', 'Mileage', and 'Power'.

**2. Filter:** Rows where Price > 5 were filtered.

**3. Rename:** The column Car_Age was renamed to Age_of_Car.

**4. Arrange:** The data was sorted by Price in descending order.

**5. Summarize:** The average price was calculated for each Location.

In [44]:
# Select specific columns
selected_df = df[['Name', 'Location', 'Price', 'Car_Age', 'Mileage', 'Power']]
selected_df.head()

,Name,Location,Price,Car_Age,Mileage,Power
0,Hyundai Creta 1.6 CRDi SX Option,Pune,12.50,9,19.67,126.20
1,Honda Jazz V,Chennai,4.50,13,13.00,88.70
2,Maruti Ertiga VDI,Chennai,6.00,12,20.77,88.76
3,Audi A4 New 2.0 TDI Multitronic,Coimbatore,17.74,11,15.20,140.80
4,Nissan Micra Diesel XV,Jaipur,3.50,11,23.08,63.10


In [47]:
# Filter rows where Price is greater than 5
filtered_df = selected_df[selected_df['Price'] < 10]
filtered_df.head()

,Name,Location,Price,Car_Age,Mileage,Power
1,Honda Jazz V,Chennai,4.50,13,13.00,88.70
2,Maruti Ertiga VDI,Chennai,6.00,12,20.77,88.76
4,Nissan Micra Diesel XV,Jaipur,3.50,11,23.08,63.10
6,Volkswagen Vento Diesel Comfortline,Pune,5.20,11,20.54,103.60
7,Tata Indica Vista Quadrajet LS,Chennai,1.95,12,22.30,74.00


In [48]:
# Rename columns (e.g., rename 'Car_Age' to 'Age_of_Car')
renamed_df = filtered_df.rename(columns={'Car_Age': 'Age_of_Car'})
renamed_df.head()

,Name,Location,Price,Age_of_Car,Mileage,Power
1,Honda Jazz V,Chennai,4.50,13,13.00,88.70
2,Maruti Ertiga VDI,Chennai,6.00,12,20.77,88.76
4,Nissan Micra Diesel XV,Jaipur,3.50,11,23.08,63.10
6,Volkswagen Vento Diesel Comfortline,Pune,5.20,11,20.54,103.60
7,Tata Indica Vista Quadrajet LS,Chennai,1.95,12,22.30,74.00


In [49]:
# Arrange/sort by 'Price' in descending order
arranged_df = renamed_df.sort_values(by='Price', ascending=False)
arranged_df.head()

,Name,Location,Price,Age_of_Car,Mileage,Power
1873,Mercedes-Benz S Class 2005 2013 S 350 L,Mumbai,9.99,17,10.13,364.90
3602,Ford EcoSport 1.5 Petrol Titanium Plus AT,Pune,9.99,7,14.80,121.36
3651,Mercedes-Benz E-Class 280 CDI,Hyderabad,9.99,15,11.00,198.50
1850,Maruti Ciaz 1.4 Zeta,Kochi,9.96,6,20.73,91.10
931,Audi A4 2.0 TDI,Kolkata,9.95,15,16.55,147.51


In [50]:
# Summarize: Group by 'Location' and calculate average price
summary_df = arranged_df.groupby('Location').agg({'Price': 'mean'}).reset_index()
summary_df.head()

,Location,Price
0,Ahmedabad,4.910566
1,Bangalore,5.135161
2,Chennai,4.295209
3,Coimbatore,6.077283
4,Delhi,4.697480


**Arranged Data:** Top 5 rows of the most expensive cars.

**Summary:** The average price by location.